# Denoise Audio — Folder Batch

Removes background noise from speech audio using **resemble-enhance**.
Processes one file at a time to minimize peak RAM.

**GPU recommended** but works on CPU. Resume-safe.

---

> Uses `resemble-enhance` + `torchaudio`.

⚠️ This step is compute-intensive. Expect ~2–5× realtime on GPU, slower on CPU.

In [ ]:
import os

# ════════════════════════════════════════════
# ⚙️  CONFIGURATION
# ════════════════════════════════════════════
os.environ.setdefault("INPUT_DIR",   "/data/audio_in")
os.environ.setdefault("OUTPUT_DIR",  "/data/audio_denoised")
os.environ.setdefault("DEVICE",      "cuda")    # cuda | cpu

INPUT_DIR  = os.environ["INPUT_DIR"]
OUTPUT_DIR = os.environ["OUTPUT_DIR"]
DEVICE     = os.environ["DEVICE"]

print(f"Input:  {INPUT_DIR}")
print(f"Output: {OUTPUT_DIR}")
print(f"Device: {DEVICE}")

In [ ]:
!pip install --quiet --break-system-packages resemble-enhance torchaudio tqdm

In [ ]:
import os
import torch
import torchaudio
from pathlib import Path
from tqdm import tqdm
from resemble_enhance.enhancer.inference import denoise

os.makedirs(OUTPUT_DIR, exist_ok=True)

files = sorted(Path(INPUT_DIR).rglob("*.wav"))
print(f"Found {len(files)} WAV files")

done, skipped, failed = 0, 0, 0
for src in tqdm(files, desc="Denoising", unit="file"):
    rel = src.relative_to(INPUT_DIR)
    dst = Path(OUTPUT_DIR) / rel
    dst.parent.mkdir(parents=True, exist_ok=True)

    if dst.exists():
        skipped += 1
        continue

    try:
        wav, sr = torchaudio.load(str(src))
        # Mix to mono if needed
        if wav.shape[0] > 1:
            wav = wav.mean(dim=0)
        else:
            wav = wav.squeeze(0)

        denoised, new_sr = denoise(wav, sr, DEVICE)
        torchaudio.save(str(dst), denoised.unsqueeze(0).cpu(), new_sr)
        done += 1
    except Exception as e:
        tqdm.write(f"  FAIL {src.name}: {e}")
        failed += 1

# Free GPU memory
if DEVICE == "cuda":
    try:
        torch.cuda.empty_cache()
    except Exception:
        pass

print(f"\nDone: {done} denoised, {skipped} skipped, {failed} failed")